# Лабораторная работа 1. Подсчет ошибки распознавания

WER (Word Error Rate) является метрикой для оценки качества систем распознавания речи. Она показывает процент ошибочных слов в гипотезе распознавания по сравнению с эталонным текстом. WER учитывает три типа ошибок: вставки, удаления и замены. 
$$ WER = {I + D + S \over D + S + C} $$
где I, D, S - количество втавок, удалений и замен, соответственно. C - количество правильно распознанных слов

Лабораторная работа состоит из трех частей. Первая часть (функция подсчета WER) обязательная, остальные дополнительные. Всего за работу можно получить максимум 20 баллов. 4 за сдачу в срок и 16 за задания: 
* функция подсчета WER (тест 1.a, 1.b) - 8 баллов
* функция подсчета WER и ошибки пунктуации (тест 2.a) - 4 балла
* функция подсчета SA-WER (тест 3.а) - 4 балла

# 1. Word Error Rate (8 баллов)

## 1.a. подсчет WER 


Функция должна принимать две строки в качестве входных данных: эталонный текст и распознанный текст. Эталонный текст - это то, что произносится в аудиозаписи, а гипотеза распознавания - это текст, полученный от системы распознавания речи. Для корректного вычисления ошибки распознавания необходимо удалить все символы пунктуации и привести все слова к нижнему регистру.



In [30]:
import string

In [3]:
def calculate_wer(reference_text: str, recognized_text: str) -> float:
    # Приведение текста к нижнему регистру, удаление символов пунктуации и разбивка на слова
    reference_words = reference_text.lower().translate(str.maketrans('', '', string.punctuation)).split()
    recognized_words = recognized_text.lower().translate(str.maketrans('', '', string.punctuation)).split()

    # Инициализация матрицы для подсчета расстояния между словами
    distance_matrix = [[0] * (len(recognized_words) + 1) for _ in range(len(reference_words) + 1)]

    # Наполнение первой строки матрицы
    for i in range(len(reference_words) + 1):
        distance_matrix[i][0] = i

    # Наполнение первого столбца матрицы
    for j in range(len(recognized_words) + 1):
        distance_matrix[0][j] = j

    # Заполнение матрицы расстояний методом динамического программирования
    for i in range(1, len(reference_words) + 1):
        for j in range(1, len(recognized_words) + 1):
            substitution_cost = 0 if reference_words[i - 1] == recognized_words[j - 1] else 1
            distance_matrix[i][j] = min(
                distance_matrix[i - 1][j] + 1,  # удаление
                distance_matrix[i][j - 1] + 1,  # вставка
                distance_matrix[i - 1][j - 1] + substitution_cost  # замена
            )

    # Расчет WER (в процентах)
    wer = (distance_matrix[len(reference_words)][len(recognized_words)] / len(reference_words)) * 100
    return wer

wer = calculate_wer('Я ел солонину', 'Я ел слона')
print(f"Word Error Rate: {wer:.2f}%")

Word Error Rate: 33.33%


In [4]:
def assert_wer(ref, hyp, ideal_wer):
    wer = calculate_wer(ref, hyp)
    assert round(wer, 2) == round(ideal_wer, 2), f"for '{hyp=}' and '{ref=}' {ideal_wer=}, calculate_wer {wer=}"
    
def test_wer():
    assert_wer('привет студент', 'привет студент', 0)
    assert_wer('привет! Студент.', 'Привет, студент?', 0)
    assert_wer('привет студент', 'студент', 50)
    assert_wer('привет студент', '', 100)
    assert_wer('привет студент', 'студент привет', 100)
    assert_wer('привет', 'привет студент', 100)
    assert_wer('привет студент привет как дела', 'студент привет', 60)
    assert_wer('привет студент привет как дела', 'привет как дела', 40)
    assert_wer('привет студент привет как дела ', 'привет студент дела ', 40)
    assert_wer('привет студент привет как дела '*100, 'привет студент дела '*100, 40)

    print(f"Test 1.a passed")
    
test_wer() 
    

Test 1.a passed


## 1.b. Построение выравнивания
Реализованная в части 1.a. функция выдает только суммарное значение ошибки распознавания, не давая понимания, в чем состоят основные проблемы распознавания. 

Значение WER получается из трех видов ошибок: 
* вставка (insertion)
* удаление (deletion)
* замена (substitution)

Каждый тип ошибок имеет свое значение и указывает на определенные недостатки системы. Например, большое количество вставок означает, что ASR слышит речь там где ее нет. А большое количество удалений показывает, что целевая речь пропускается и не транскрибируется. 

Кроме числовых значений каждой ошибки, для анализа результатов работы системы может пригодиться выравнивание эталонной текстовки и гипотезы распознавания относительно друг друга. 

пример выравнивания: 

```
>>> tabulate(ali)

Я сегодня  ***   учуcь  в  универе
Я    с    завтра учусь *** универе  
C    S      I      C    D    C    
```

Реализуйте функцию, которая кроме числового значения WER возвращает выравнивание, а также значения каждого типа ошибок распознавания (вставки, удаления, замены).

In [1]:
!pip --version

pip 23.3.1 from /home/ext-pishchako-a@ad.speechpro.com/Pishchako_A/my_venvs/itmo39/lib/python3.9/site-packages/pip (python 3.9)


In [3]:
#!pip install tabulate
from tabulate import tabulate
# используйте tabulate для отладки

In [276]:
def calculate_wer_with_alignment(reference_text: str, recognized_text: str):
    # Приведение текста к нижнему регистру, удаление символов пунктуации и разбивка на слова
    reference_words = reference_text.lower().translate(str.maketrans('', '', string.punctuation)).split()
    recognized_words = recognized_text.lower().translate(str.maketrans('', '', string.punctuation)).split()

    # Инициализация матрицы для подсчета расстояния между словами
    distance_matrix = [[0] * (len(recognized_words) + 1) for _ in range(len(reference_words) + 1)]

    # Наполнение первой строки матрицы
    for i in range(len(reference_words) + 1):
        distance_matrix[i][0] = i

    # Наполнение первого столбца матрицы
    for j in range(len(recognized_words) + 1):
        distance_matrix[0][j] = j

    # Заполнение матрицы расстояний методом динамического программирования
    for i in range(1, len(reference_words) + 1):
        for j in range(1, len(recognized_words) + 1):
            substitution_cost = 0 if reference_words[i - 1] == recognized_words[j - 1] else 1
            distance_matrix[i][j] = min(
                distance_matrix[i - 1][j] + 1,  # удаление
                distance_matrix[i][j - 1] + 1,  # вставка
                distance_matrix[i - 1][j - 1] + substitution_cost  # замена
            )

    # Восстановление выравнивания
    i, j = len(reference_words), len(recognized_words)
    alignment_reference = []
    alignment_recognized = []
    alignment_annotation = []

    while i > 0 or j > 0:
            if i > 0 and j > 0 and reference_words[i - 1] == recognized_words[j - 1] and distance_matrix[i][j] == distance_matrix[i - 1][j - 1]:
                alignment_reference.append(reference_words[i - 1])
                alignment_recognized.append(recognized_words[j - 1])
                alignment_annotation.append("C")
                i -= 1
                j -= 1
            elif i > 0 and (j > 0 and distance_matrix[i][j]  == distance_matrix[i - 1][j - 1] + 1):
                alignment_reference.append(reference_words[i - 1])
                alignment_recognized.append(recognized_words[j - 1])
                alignment_annotation.append("S")
                i -= 1
                j -= 1
            elif j > 0 and (i == 0 or distance_matrix[i][j - 1] + 1 <= distance_matrix[i - 1][j] + 1):
                alignment_reference.append("***")
                alignment_recognized.append(recognized_words[j - 1])
                alignment_annotation.append("I")
                j -= 1
            elif i > 0 and (j == 0 or distance_matrix[i - 1][j] + 1 <= distance_matrix[i][j - 1] + 1):
                alignment_reference.append(reference_words[i - 1])
                alignment_recognized.append("***")
                alignment_annotation.append("D")
                i -= 1

    alignment_reference.reverse()
    alignment_recognized.reverse()
    alignment_annotation.reverse()

    # Подсчет операций
    correct = sum(1 for a, ref, rec in zip(alignment_annotation, alignment_reference, alignment_recognized) if a == "C" and (ref != "***" or rec != "***"))
    deletion = sum(1 for a, ref in zip(alignment_annotation, alignment_reference) if a == "D" and ref != "***")
    insertion = sum(1 for a, rec in zip(alignment_annotation, alignment_recognized) if a == "I" and rec != "***")
    substitution = sum(1 for a, rec, ref in zip(alignment_annotation, alignment_recognized, alignment_reference) if a == "S" and (rec != "***" or ref != "***"))

    # Поправка на общее количество слов в референсе
    total_reference_words = sum(1 for w in reference_words if w != '***')
    wer = int((deletion + insertion + substitution) / total_reference_words * 100)

    # Включение информации о выравнивании в результат
    alignment_result = [
        alignment_reference,
        alignment_recognized,
        alignment_annotation
    ]

    return {"wer": wer, "cor": correct, "del": deletion, "ins": insertion, "sub": substitution, "ali": alignment_result}

In [277]:
# def calculate_wer_with_alignment(reference_text: str, recognized_text: str):
#     return {
#             "wer" : 0,
#             "cor": 2, 
#             "del": 0,
#             "ins": 0,
#             "sub": 0,
#             "ali": [["привет", "студент"],["привет", "студент"],['C', 'C']]}

In [278]:
def assert_wer_with_alignment(ref, hyp, ideal_report):
    report = calculate_wer_with_alignment(ref, hyp)
    for k, v in ideal_report.items():
        if isinstance(v, float):
            assert round(v, 2) == round(report[k], 2), f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"
        else:
            assert v == report[k], f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"

    
def test_wer_with_alignment():
    assert_wer_with_alignment('привет студент', 'привет студент',  {
            "wer" : 0,
            "cor": 2, 
            "del": 0,
            "ins": 0,
            "sub": 0,
            "ali": [["привет", "студент"],["привет", "студент"],['C', 'C']]})
    assert_wer_with_alignment('привет студент', 'студент', {
            "wer" : 50,
            "cor": 1, 
            "del": 1,
            "ins": 0,
            "sub": 0,
            "ali": [["привет", "студент"],["***", "студент"],['D', 'C']]})
    assert_wer_with_alignment('привет', 'привет студент', {
            "wer" : 100,
            "cor": 1, 
            "del": 0,
            "ins": 1,
            "sub": 0,
            "ali": [["привет", "***"],["привет", "студент"],['C', 'I']]})
    assert_wer_with_alignment('привет студент', 'пока студент',  {
            "wer" : 50,
            "cor": 1, 
            "del": 0,
            "ins": 0,
            "sub": 1,
            "ali": [["привет", "студент"],["пока", "студент"],['S', 'C']]})

    print(f"Test 1.b passed")
    
test_wer_with_alignment() 

Test 1.b passed


# 2. WER с пунктуацией (4 балла)
Попробуйте модифицировать WER таким образом, чтобы получившаяся метрика учитавала ошибки расстановки знаков препинания. 

Для этого надо ввести ограничение в алгоритм подсчета distance_matrix таким образом, чтобы запретить делать замену знака препинания на слово и наоборот.

Пример выравнивания 
```
Я сегодня  .   ***   ***  А ты  
Я    с    *** завтра  ?   А ты  
C    S    D_p   I    I_p  C  C    
```
Здесь суффикс _p в аннотации к ошибкам означает **ошибки пунктуации**


Задание: 
Напишите функцию, которая кроме стандартного WER считает дополнительно PunctuaionErrorRate (PER) по формуле

$$ PER = {I_p + D_p + S_p \over D_p + S_p + C_p} $$


In [368]:
def calculate_wer_per(reference_text: str, recognized_text: str):
    distance_matrix = []
    
    punct_list = list("!\"#$%&'()*+, -./:;<=>?@[\]^_`{|}~")  # Список знаков пунктуации
    
    # Приведение текста к нижнему регистру, удаление символов пунктуации и разбивка на слова
    reference_words = preproc(reference_text)
    recognized_words = preproc(recognized_text)
    
    # Инициализация матрицы для подсчета расстояния между словами
    distance_matrix = [[0] * (len(recognized_words) + 1) for _ in range(len(reference_words) + 1)]
    
    # Наполнение первой строки матрицы
    for i in range(len(reference_words) + 1):
        distance_matrix[i][0] = i
    
    # Наполнение первого столбца матрицы
    for j in range(len(recognized_words) + 1):
        distance_matrix[0][j] = j
    
    # Заполнение матрицы расстояний методом динамического программирования
    for i in range(1, len(reference_words) + 1):
        for j in range(1, len(recognized_words) + 1):
            if reference_words[i - 1] == recognized_words[j - 1]:
                cost = 0
            else:
                cost = 1
            
            insertion = distance_matrix[i - 1][j] + 1
            deletion = distance_matrix[i][j - 1] + 1
            
            if reference_words[i-1] in punct_list and recognized_words[j-1] in punct_list \
                or reference_words[i-1] not in punct_list and recognized_words[j-1] not in punct_list: 
                    substitution = distance_matrix[i-1][j-1] + cost
                    distance_matrix[i][j] = min(insertion, deletion, substitution)
            else:
                distance_matrix[i][j] = min(insertion, deletion)

    # Восстановление выравнивания
    i, j = len(reference_words), len(recognized_words)
    steps = []
    
    while i > 0 or j > 0:
        if i > 0 and distance_matrix[i][j] == distance_matrix[i-1][j] + 1:       
            if reference_words[i-1] in punct_list:
                steps.append('D_p')
            else:
                steps.append('D')
            i = i - 1
        
        elif j > 0 and distance_matrix[i][j] == distance_matrix[i][j-1] + 1:
            if recognized_words[j-1] in punct_list:
                steps.append('I_p')
            else:
                steps.append('I')
            j = j - 1
        
        else:
            if reference_words[i-1] == recognized_words[j-1]:
                if reference_words[i-1] in punct_list and recognized_words[j-1] in punct_list:
                    steps.append('C_p')
                else:
                    steps.append('C')
            else:
                if reference_words[i-1] in punct_list:
                    steps.append('S_p')
                else:
                    steps.append('S')
            i = i - 1
            j = j - 1
    
    steps = steps[::-1]
    
    # (I + D + S) / (D + S + C)
    WER = len([cur for cur in steps if cur == 'I' or cur == 'D' or cur == 'S']) \
        / len([cur for cur in steps if cur == 'D' or cur == 'S' or cur == 'C'])
    
    # (IP + DP + SP) / (DP + SP + CP)
    DSC_p = len([cur for cur in steps if cur == 'D_p' or cur == 'S_p' or cur == 'C_p'])
    PER = len([cur for cur in steps if cur == 'I_p' or cur == 'D_p' or cur == 'S_p']) \
        / len([cur for cur in steps if cur == 'D_p' or cur == 'S_p' or cur == 'C_p']) if DSC_p != 0 else 0

    for ind, step in enumerate(steps):
        if step == 'I' or step == 'I_p':
            reference_words.insert(ind, "***")
        elif step == 'D' or step == 'D_p':
            recognized_words.insert(ind, "***")

    ali = []
    ali.append(reference_words)
    ali.append(recognized_words)
    ali.append(steps)

    correct = steps.count('C')
    deletion = steps.count('D')
    insertion = steps.count('I')
    substitution = steps.count('S')

    wer = int(WER * 100)
    per = int(PER * 100)

    return {"wer": wer,
            "per": per}

In [369]:
def assert_wer_per(ref, hyp, ideal_report):
    report = calculate_wer_per(ref, hyp)
    for k, v in ideal_report.items():
        if isinstance(v, float):
            assert round(v, 2) == round(report[k], 2), f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"
        else:
            assert v == report[k], f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"
    
def test_wer_per():
    assert_wer_per('привет студент.', 'привет студент',  {
            "wer" : 0,
            "per": 100})
    assert_wer_per('привет студент.', 'студент.', {
            "wer" : 50,
            "per": 0,})
    assert_wer_per('привет студент.', 'привет. студент',  {
            "wer" : 0,
            "per": 200})
    assert_wer_per('привет студент.', '.студент?', {
            "wer" : 50,
            "per": 200, })

    print(f"Test 2 passed")
    
test_wer_per() 

Test 2 passed


# 3. Speaker-attributed Word Error Rate (4 балла)

В задаче распознавания диалоговых данных, когда говорят два диктора, важно не только распознать правильно каждое слово, но и отнести его к правильному диктору. Чаще всего такой результат получается с помощью комбинации двух независимых систем: распознавания речи и диаризация. 

При подсчете ошибки распознавания диалоговых систем в формулу WER добавляется еще один тип ошибки - S_I (speaker incorrect).

$$ SA{\text -}WER = \min{I + D + S + S_I \over D + S + C + S_I} $$

Кроме подсчета самой ошибки, sa-wer решает еще одну задачку - поиск маппинга из эталонных названий спикеров (например, имен) в предсказанные (чаще всего Idшники). Это необходимо, тк система диаризации не знает, какие названия у спикеров в эталоне. При подсчете SA-WER проверяются все возможные мапинги спикеров и выбирается тот, который соответствует минимальному значению ошибки. 



In [406]:
import copy
import itertools

def calculate_sawer(reference_text, reference_speakers, recognized_text, recognized_speakers):
    assert isinstance(reference_text, list)
    assert isinstance(recognized_text, list)
    assert len(reference_text) == len(reference_speakers)
    assert len(recognized_text) == len(recognized_speakers)

    reference_words = copy.deepcopy(reference_text)
    recognized_words = copy.deepcopy(recognized_text)

    # Расстояние Левенштейна
    distance_matrix = [[0] * (len(recognized_words) + 1) for _ in range(len(reference_words) + 1)]

    for i in range(len(reference_words) + 1):
        distance_matrix[i][0] = i

    for j in range(len(recognized_words) + 1):
        distance_matrix[0][j] = j

    for i in range(1, len(reference_words) + 1):
        for j in range(1, len(recognized_words) + 1):
            cost = 0 if reference_words[i - 1] == recognized_words[j - 1] else 1
            insertion = distance_matrix[i - 1][j] + 1
            deletion = distance_matrix[i][j - 1] + 1
            substitution = distance_matrix[i - 1][j - 1] + cost
            distance_matrix[i][j] = min(insertion, deletion, substitution)

    # Расчет WER (в процентах)
    wer = (distance_matrix[-1][-1] / len(reference_words)) * 100

    # Восстановление пути и учет ошибок в атрибуции спикеров
    i = len(reference_words)
    j = len(recognized_words)

    steps = []
    ref_speaker_list = []
    rec_speaker_list = []

    while i > 0 or j > 0:
        if i >= 1 and distance_matrix[i][j] == distance_matrix[i - 1][j] + 1:
            steps.append('D')
            i = i - 1

        elif j >= 1 and distance_matrix[i][j] == distance_matrix[i][j - 1] + 1:
            steps.append('I')
            j = j - 1

        else:
            if reference_words[i - 1] == recognized_words[j - 1]:
                steps.append('C')
                ref_speaker_list.append(reference_speakers[i - 1])
                rec_speaker_list.append(recognized_speakers[j - 1])
            else:
                steps.append('S')

            i = i - 1
            j = j - 1

    steps = steps[::-1]
    ref_speaker_list = ref_speaker_list[::-1]
    rec_speaker_list = rec_speaker_list[::-1]

    # Создание наборов уникальных дикторов для эталонных и распознанных текстов
    unique_ref_speakers = list(set(ref_speaker_list))
    unique_rec_speakers = list(set(rec_speaker_list))

    # Генерация перестановок уникальных дикторов для распознанных текстов
    permutations = itertools.permutations(unique_rec_speakers)

    # Инициализация счётчика неправильно распознанных дикторов большим значением
    incorrect_speakers = len(ref_speaker_list) + len(rec_speaker_list) + 20

    for permutation in permutations:
        errors = 0

        for i, reference in enumerate(ref_speaker_list):
            if reference in unique_ref_speakers and permutation[unique_ref_speakers.index(reference)] != rec_speaker_list[i]:
                errors += 1

        if errors < incorrect_speakers:
            incorrect_speakers = errors

    # Вычисление SA-WER
    total_words = len(reference_words)
    if total_words == 0:
        sawer = 0
    else:
        sawer = wer + (incorrect_speakers / total_words * 100)
    
    return {"sawer": sawer}

In [407]:
def assert_sawer(reference_text, reference_speakers, recognized_text, recognized_speakers, ideal_report):
    report = calculate_sawer(reference_text, reference_speakers, recognized_text, recognized_speakers)
    for k, v in ideal_report.items():
        assert v == report[k]

    
def test_sawer():
    assert_sawer(['привет', 'студент'], ['A', 'B'], ['привет', 'студент'], [1, 2],  {
            "sawer" : 0})
    assert_sawer(['привет', 'студент'], ['A', 'A'], ['привет', 'студент'], [1, 2],  {
            "sawer" : 50})
    assert_sawer(['привет', 'студент'], ['A', 'A'], ['привет', 'студент'], [0, 0],  {
            "sawer" : 0})
    assert_sawer(['привет', 'с'], ['A', 'B'], ['привет', 'студент'], [1, 2],  {
            "sawer" : 50})
    assert_sawer(['привет', 'с'], ['A', 'B'], ['привет'], [1],  {
            "sawer" : 50})
    assert_sawer(['привет'], ['A'], ['привет', 'студент'], [1, 0],  {
            "sawer" : 100})
    assert_sawer(['привет'], ['A'], ['привет', 'студент'], [0, 0],  {
            "sawer" : 100})

    print(f"Test 3 passed")
    
test_sawer()

Test 3 passed
